In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import nltk
import re
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from sklearn.externals import joblib
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
df = pd.read_pickle('../data/datagroups.pkl')

In [4]:
df.shape

(2811774, 13)

In [9]:
groupDF = df.groupby('group_id').tweet_id.count().reset_index()
groupDF.sort_values('tweet_id',ascending=False, inplace=True)

In [10]:
groupDF

,group_id,tweet_id
325140,group_392623,1339
769545,group_792589,653
781819,group_88734,448
381313,group_443179,400
508003,group_55720,284
581994,group_623792,231
357041,group_421333,224
723431,group_751085,220
444659,group_50019,213
484018,group_535613,212


In [11]:
filteredDF = df[df.group_id == 'group_392623']

In [12]:
filteredDF

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,date,time,month,all_sent,group_id,tweet_num
1460632,1605828,492680,True,Mon Nov 06 00:00:42 +0000 2017,@492681 @492679 @129141 @135779 @ATVIAssist I ...,1605829,1605827.0,2017-11-06 00:00:42,00:00:42,11,0.0000,group_392623,1
1461573,1606076,140141,True,Mon Nov 06 00:02:05 +0000 2017,@492680 @ATVIAssist @115765 J ai le même problème,1606079,625790.0,2017-11-06 00:02:05,00:02:05,11,0.0000,group_392623,2
1460633,1605829,492679,True,Mon Nov 06 00:02:22 +0000 2017,@492680 @492681 @129141 @135779 @ATVIAssist Ye...,1605830,1605828.0,2017-11-06 00:02:22,00:02:22,11,0.5574,group_392623,3
1461607,1606117,493355,True,Wed Nov 08 00:04:04 +0000 2017,@493354 @492680 @493357 @493307 @493356 @49335...,"1606118,1606119",1606115.0,2017-11-08 00:04:04,00:04:04,11,-0.4019,group_392623,4
1461060,625387,492983,True,Mon Nov 06 00:05:00 +0000 2017,"@ATVIAssist Around 11am EST it was back up, th...",NaN,625011.0,2017-11-06 00:05:00,00:05:00,11,0.8555,group_392623,5
1461059,625386,492982,True,Mon Nov 06 00:05:08 +0000 2017,@ATVIAssist Haven’t been able to connect and h...,NaN,625011.0,2017-11-06 00:05:08,00:05:08,11,0.3400,group_392623,6
1461058,625385,492981,True,Mon Nov 06 00:05:35 +0000 2017,@ATVIAssist @115765 Not working again,NaN,625011.0,2017-11-06 00:05:35,00:05:35,11,0.0000,group_392623,7
1461057,625384,492980,True,Mon Nov 06 00:05:46 +0000 2017,@ATVIAssist @115765 Does support even answer p...,NaN,625011.0,2017-11-06 00:05:46,00:05:46,11,0.4019,group_392623,8
1461056,625383,492979,True,Mon Nov 06 00:06:11 +0000 2017,@ATVIAssist I’m having trouble,NaN,625011.0,2017-11-06 00:06:11,00:06:11,11,-0.4019,group_392623,9
1461574,1606079,492680,True,Mon Nov 06 00:06:37 +0000 2017,@140141 @ATVIAssist @115765 Same,NaN,1606076.0,2017-11-06 00:06:37,00:06:37,11,0.0000,group_392623,10


Looks like there are interactions, mainly for activision

In [15]:
import plotly.plotly as py
from plotly.graph_objs import *

import networkx as nx